In [ ]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

In [ ]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
#APPEND to Mongo
scraped_data = {}

############################################################################

#First go to NASA news
url = "https://mars.nasa.gov/news/"
browser.visit(url)

time.sleep(1)

soup = BeautifulSoup(browser.html)

slide = soup.find("li", {"class": "slide"})
news_title = slide.find("div", {"class": "content_title"}).text
news_p = slide.find("div", {"class": "article_teaser_body"}).text.strip()

# I got stuck on this one. The link in the HW did not return a clickable image. Only a searchable gallery.
base = "https://www.jpl.nasa.gov"
url = f"{base}/images/supercams-mars-meteorite-aboard-the-iss/" # This was the current featured image. I reviewed the office hours recording and chose to navigate to the featured gallery image.
browser.visit(url)
time.sleep(1)

browser.find_by_id("82498").click()
time.sleep(1)

soup = BeautifulSoup(browser.html)
image = soup.find("img", {"class": "82498"})

In [ ]:
#clickable image at the end of this code
#SOURCE:https://stackoverflow.com/questions/56943210/how-to-get-src-attribute-from-image-with-python
featured_image_url = soup.findAll('img')[2]
print(featured_image_url['src'])

In [ ]:
#Mars Facts
url = "https://space-facts.com/mars/"
browser.visit(url)
time.sleep(1)

data = pd.read_html(browser.html)

data1 = data[0]
data1.columns = ["Stats", "Value"]
mars_facts = data1.to_html(index=False)

# EXIT browser
browser.quit()

#APPEND data
scraped_data["news_title"] = news_title
scraped_data["news_p"] = news_p
scraped_data["featured_image_url"] = featured_image_url
scraped_data["mars_facts"] = mars_facts

In [ ]:
base = "https://astrogeology.usgs.gov"
url = f"{base}/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
time.sleep(1)

soup = BeautifulSoup(browser.html)
links = soup.find("div", {"class": "results"}).findAll("a", {"class": "itemLink"})

#FILTER non images
realLinks = []
for link in links:
    image = link.find("img")
    if (image):
        realLinks.append(base + link["href"]) 
        # APPEND base url

# LOOP thru each img link, click & gather info
hemisphere_data = []
for realLink in realLinks:
    browser.visit(realLink)
    time.sleep(1)
    
    soup = BeautifulSoup(browser.html)
    hemi_url = soup.find("ul").find("li").find("a")["href"]
    hemi_title = soup.find("h2", {'class', "title"}).text.split(" Enhanced")[0]
    
    hemisphere_data.append({"title": hemi_title, "url": hemi_url})

#EXIT browser
browser.quit()

#APPEND data
scraped_data["news_title"] = news_title
scraped_data["news_p"] = news_p
scraped_data["featured_image_url"] = featured_image_url
scraped_data["mars_facts"] = mars_facts
scraped_data["hemispheres"] = hemisphere_data

scraped_data